# Timescaledb example

In [1]:
import psycopg2 as psy
from datetime import datetime, timedelta

In [2]:
# create testdb
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

con = psy.connect("dbname=postgres host=localhost user=postgres password=foobar")

with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("DROP DATABASE IF EXISTS testdb;")
        cur.execute('CREATE DATABASE testdb')

# not sure if the AUTOCOMMIT is needed
with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;")
con.close()

In [3]:
con = psy.connect("dbname=testdb host=localhost user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
        CREATE TABLE conditions (
          time        TIMESTAMPTZ       NOT NULL,
          location    TEXT              NOT NULL,
          temperature DOUBLE PRECISION  NULL,
          humidity    DOUBLE PRECISION  NULL
        );""")


In [4]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT create_hypertable('conditions', 'time');")

In [5]:
SQL = "INSERT INTO conditions(time, location, temperature, humidity) VALUES (%s, %s, %s, %s)"
tstart = datetime.now()
tnow = tstart
delta = timedelta(seconds=60)
with con:
    for i in range(1000):
        with con.cursor() as cur:
            cur.execute(SQL, (tnow, "here", i%20, i%35))
        tnow = tnow + delta

In [6]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT * FROM conditions ORDER BY time DESC LIMIT 10;")
        for x in cur.fetchall():
            print(x)

(datetime.datetime(2019, 1, 23, 16, 10, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 19.0, 19.0)
(datetime.datetime(2019, 1, 23, 16, 9, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 18.0, 18.0)
(datetime.datetime(2019, 1, 23, 16, 8, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 17.0, 17.0)
(datetime.datetime(2019, 1, 23, 16, 7, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 16.0, 16.0)
(datetime.datetime(2019, 1, 23, 16, 6, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 15.0, 15.0)
(datetime.datetime(2019, 1, 23, 16, 5, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 14.0, 14.0)
(datetime.datetime(2019, 1, 23, 16, 4, 52, 828023, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'here', 13.0, 13.0)
(datetime.datetime(2019, 1, 23, 16, 3, 52, 828023, tzinfo=psycopg2.t

In [24]:
SQL= """SELECT time_bucket('5 minute', time) AS five_min, count(*)
         FROM conditions WHERE time > '%s' GROUP BY five_min ORDER BY five_min LIMIT 5;"""

with con:
    with con.cursor() as cur:
        # cur.execute(SQL, (tstart))
        cur.execute(SQL % tstart)
        for x in cur.fetchall():
            print(x)

(datetime.datetime(2019, 1, 22, 23, 30, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 3)
(datetime.datetime(2019, 1, 22, 23, 35, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 5)
(datetime.datetime(2019, 1, 22, 23, 40, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 5)
(datetime.datetime(2019, 1, 22, 23, 45, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 5)
(datetime.datetime(2019, 1, 22, 23, 50, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 5)
